#1. Thêm thư viện

In [30]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import cv2
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Convolution2D, MaxPooling2D, Flatten
from tensorflow.keras import utils
import tensorflow as tf
from tensorflow import keras


#2. Đường dẫn và mảng lưu trữ tên lớp

In [31]:
path = "Class/"
categories = ['Audi', 'hyundai', 'lexus', 'mazda', 'Mercedes', 'toyota', 'volkswagen','opel']


#3. Khởi tạo dữ liệu và lớp

In [32]:
data = []       #
labels = []     #Lưu chỉ số lớp
imagePaths = [] #Lưu trữ đường dẫn ảnh và chỉ số nhãn về nó

HEIGHT = 32
WIDTH = 32

N_CHANNELS = 3 #Kênh màu

#4.Xáo trộn đường dẫn 1 cách ngẫu nhiên lưu vào mảng "data" và đưa các hình ảnh về chung chuẩn

In [34]:
#Lưu đường dẫn và lớp và mảng imagePaths
for index ,name in enumerate(categories):
    for file in os.listdir(path+name):
        imagePaths.append([path+name+'/'+file, index])

#Xáo trộn dữ liệu
import random
random.shuffle(imagePaths)
print(imagePaths[:10])

#Đưa các hình ảnh về kích cỡ tiêu chuẩn
for imagePath in imagePaths:
    image = cv2.imread(imagePath[0])            #Đọc hình ảnh thông qua đường dẫn
    image = cv2.resize(image, (WIDTH, HEIGHT))  #resize hình ảnh
    data.append(image)

    labels.append( int(imagePath[1]) )               #Lưu chỉ số theo ngẫu nhiên


#Tiền xử lý dữ liệu
data = np.array(data, dtype='float')/255
labels = np.array(labels)

print('Labels')
print(labels)
# print('Data')
# print(data)

[['Class/Mercedes/Mercede (763).jpg', 4], ['Class/mazda/5401335763581300412_rotate_90_73.jpg', 3], ['Class/volkswagen/vw-3306790_960_720_rotate_90_263.jpg', 6], ['Class/Mercedes/mercede (502).jpg', 4], ['Class/Mercedes/mercede (594).jpg', 4], ['Class/Audi/audi (1517).jpg', 0], ['Class/lexus/images117.jpg', 2], ['Class/Mercedes/mercede (305).jpg', 4], ['Class/Mercedes/mercede (201).jpg', 4], ['Class/Audi/audi (770).jpg', 0]]
Labels
[0 5 4 ... 4 2 3]


<h2>5# Chia tập dữ liệu</h2>

In [35]:

(X_train, X_test, Y_train, Y_test) = train_test_split(data, labels, test_size=0.3, random_state=42)

#Tiền xử lý các lớp
Y_train = utils.to_categorical(Y_train, num_classes=len(categories))
print(X_train.shape)
print(X_test.shape)
print(Y_train.shape)
print(Y_test.shape)


(14231, 32, 32, 3)
(6100, 32, 32, 3)
(14231, 8)
(6100,)


#6. Các thông số quan trọng

In [36]:
EPOCH = 35  #Số lần học lại
BS = 24 #Số cụm học
INIT_LR = 1e-3

class_name = categories

#7.Mô hình kiến trúc xây dựng lenet

In [37]:
model = Sequential()
model.add(Convolution2D(filters=32, kernel_size=(5,5), padding='same', activation='relu', input_shape=(WIDTH, HEIGHT, N_CHANNELS)))
model.add(MaxPooling2D(strides=2))
model.add(Convolution2D(filters=32, kernel_size=(5,5), padding='valid', activation='relu'))
model.add(Flatten())        #Chuyền từ 2D or 3D sang 1D
model.add(Dense(256, activation='relu'))
model.add(Dense(84, activation='relu'))
model.add(Dense(len(categories), activation='softmax'))

opt = tf.keras.optimizers.legacy.Adam(learning_rate=INIT_LR, decay=INIT_LR/EPOCH )
model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])
print(model.summary())

model.fit(X_train, Y_train, batch_size=BS, epochs=EPOCH, verbose=1)

Epoch 1/35


593/593 [==============================] - 26s 37ms/step - loss: 1.4920 - accuracy: 0.4246
Epoch 2/35
593/593 [==============================] - 21s 36ms/step - loss: 0.8751 - accuracy: 0.6952
Epoch 3/35
593/593 [==============================] - 21s 35ms/step - loss: 0.3948 - accuracy: 0.8708
Epoch 4/35
593/593 [==============================] - 21s 35ms/step - loss: 0.1767 - accuracy: 0.9459
Epoch 5/35
593/593 [==============================] - 22s 37ms/step - loss: 0.1018 - accuracy: 0.9701
Epoch 6/35
593/593 [==============================] - 22s 37ms/step - loss: 0.0679 - accuracy: 0.9807
Epoch 7/35
593/593 [==============================] - 22s 37ms/step - loss: 0.0397 - accuracy: 0.9889
Epoch 8/35
593/593 [==============================] - 21s 35ms/step - loss: 0.0592 - accuracy: 0.9821
Epoch 9/35
593/593 [==============================] - 22s 37ms/step - loss: 0.0422 - accuracy: 0.9878
Epoch 10/35
593/593 [==============================] - 25s 43ms/step - loss: 0.0

In [44]:
from tensorflow.keras.applications import InceptionV3, MobileNet, VGG16, DenseNet121, EfficientNetB7#, DensenNet#EfficientNet
from tensorflow.keras.layers import Dense , GlobalAveragePooling2D
from  keras import layers
from keras import models

print( " [INFO] compiling model...")
InceptionV3 = InceptionV3(input_shape=(32, 32, 3), include_top= False, weights= 'imagenet')
for layer in InceptionV3.layers:
    layer.trainable = False
model = Sequential()
model.add(InceptionV3)
model.add(GlobalAveragePooling2D())
model.add(layers.Dropout(0.5))
model.add(layers.Flatten())
model.add(layers.Dense(len(class_name), activation='softmax'))


opt = tf.keras.optimizers.legacy.Adam(learning_rate=INIT_LR, decay=INIT_LR / EPOCH)
model.compile(loss="categorical_crossentropy", optimizer=opt, metrics= ["accuracy"] )

 [INFO] compiling model...


ValueError: Input size must be at least 75x75; Received: input_shape=(32, 32, 3)

#8. Lưu mô hình và đánh giá mô hình

In [43]:
model.save_weights('inceptionV3.h5')

from numpy import argmax
from sklearn.metrics import confusion_matrix, accuracy_score

pred = model.predict(X_test)
predictions = argmax(pred, axis=1)  #trả đến nhãn

cm = confusion_matrix(Y_test, predictions) #Ma trận tương quan

fig = plt.figure()
ax = fig.add_subplot(111)
cax = ax.matshow(cm)
plt.title('Model confusion matrix')
fig.colorbar(cax)
ax.set_xticklabels(['']+ categories)
ax.set_yticklabels(['']+ categories)

for i in range(len(categories)): 
    for j in range(len(categories)):
        ax.text(i, j, cm[i,j], va='center', ha='center')

plt.xlabel('Predicted')
plt.ylabel('True')
plt.show()

ValueError: in user code:

    File "c:\Users\VTOS\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\engine\training.py", line 2440, in predict_function  *
        return step_function(self, iterator)
    File "c:\Users\VTOS\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\engine\training.py", line 2425, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "c:\Users\VTOS\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\engine\training.py", line 2413, in run_step  **
        outputs = model.predict_step(data)
    File "c:\Users\VTOS\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\engine\training.py", line 2381, in predict_step
        return self(x, training=False)
    File "c:\Users\VTOS\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\utils\traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
   

#9. Xác định độ chính xác

In [40]:
accuracy = accuracy_score(Y_test, predictions)
print('Accuracy: %.2f%%' %(accuracy*100))

Accuracy: 96.90%


#10. Đánh giá mô hình